In [1]:
import os
from dotenv import load_dotenv


# Load environment variables from the .env file
load_dotenv() 

# access the environment variables
langchain_tracing = os.getenv('LANGCHAIN_TRACING_V2')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

In [55]:
# Use Groq
#  from langchain_groq import ChatGroq
# groq_model = "llama3-8b-8192"

# model = ChatGroq(
#     temperature=0.5,
#     api_key=groq_api_key,
#     model_name=groq_model
# )


# Use OpenAI
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    temperature=0.5,
    api_key=openai_api_key,
    model_name="gpt-3.5-turbo"
)

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
        [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ba70355b-d835-4b13-97b6-96adc47601b8-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

In [42]:
# trim the messages to 65 tokens
from langchain_core.messages import  trim_messages, SystemMessage
# from langchain_openai import ChatOpenAI


trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [41]:
config = {"configurable": {"session_id": "session_7"}}

In [ ]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability in {language}."
    ),
    MessagesPlaceholder(variable_name="messages")
])

chain = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model | StrOutputParser())

In [45]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did I ask?")],
        "language": "English",
    }
)
response

'You asked what is 2 + 2.'

In [46]:
from langchain_core.runnables.history import RunnableWithMessageHistory

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [47]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response

Parent run 245a11a3-5581-425d-a9b9-60c1d7d1c16f not found for run 9152ab12-c7bc-4c62-836c-a068f11d1d52. Treating as a root run.


"I'm sorry, I don't have access to your personal information."

In [48]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response

Parent run 3a749d64-5454-4778-b69e-a1bb04a16843 not found for run 131fcbf6-7296-496b-b2f0-fb3e848e9ce2. Treating as a root run.


"You haven't asked a math problem yet. Feel free to ask any math question you have, and I'll be happy to help you with it."

In [56]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r, end=" ")

Parent run e68becec-c01a-438f-b809-ed470af17e56 not found for run 7d6ca52a-290b-41a4-8314-020ea43bdd6d. Treating as a root run.


 Hi  Todd !  Here 's  a  joke  for  you :

 Why  did  the  math  book  look  sad ?

 Because  it  had  too  many  problems !  😄  